In [1]:
import os
os.chdir('C:/myroom/')
print(os.getcwd())

C:\myroom


In [2]:
#데이터 핸들링
import pandas as pd
import numpy as np

#전처리
from sklearn.preprocessing import StandardScaler, MinMaxScaler , RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

#모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

#하이퍼 파라미터 튜닝: 보통 랜덤서치로 대략 파악한 다음 그리드 서치로 미세조정
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


#평가
from sklearn.metrics import f1_score

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
#pd.set_option('display.max_rows',None)
#pd.set_option('display.max_colwidth', None)

train= pd.read_csv("./nlp-getting-started/train.csv")
test= pd.read_csv("./nlp-getting-started/test.csv")
train.shape, test.shape
# ((7613, 5), (3263, 4))

train['target'].value_counts()
#1:실제 발생, 0: 발생하지 않음

target
0    4342
1    3271
Name: count, dtype: int64

In [4]:
X=train.iloc[:,1:4]
y=train['target']
X.head()

,keyword,location,text
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,NaN,All residents asked to 'shelter in place' are ...
3,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


In [5]:
!pip install transformers

In [6]:
!pip install ipywidgets

In [5]:
X = X.fillna('unknown')
X.isnull().sum()

keyword     0
location    0
text        0
dtype: int64

In [6]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch

# 1. 트위터 특화 BERT 로드
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. 문장 리스트를 CLS 임베딩으로 벡터화하는 함수
def get_cls_batch(texts, batch_size=16, max_len=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # 1. 텍스트 전처리: HTTP URL 제거 및 공백 제거
    # 텍스트가 아닌 숫자나 다른 타입도 고려하여 처리
    texts = [str(text).replace("http://", "").replace("https://", "").strip() for text in texts]
    
    # 2. DataLoader는 텍스트 리스트로만 동작
    all_embeddings = []
    
    # 텍스트 리스트를 DataLoader로 변환
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

    for batch in dataloader:
        # 3. 배치가 여전히 텍스트 리스트이므로, 토크나이저로 텍스트를 토크나이징
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)

        # 4. 모델에 입력하고 CLS 임베딩 추출
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS 토큰 임베딩

        all_embeddings.extend(cls_embeddings.cpu().numpy())

    return all_embeddings


In [7]:
vector1 = get_cls_batch(X['keyword'].tolist())
vector2 = get_cls_batch(X['location'].tolist())
vector3 = get_cls_batch(X['text'].tolist())

In [26]:
vector1=StandardScaler().fit_transform(vector1)
vector2=StandardScaler().fit_transform(vector2)
vector3=StandardScaler().fit_transform(vector3)

vector1.shape

(7613, 768)

In [46]:
from sklearn.decomposition import PCA

p1=pd.DataFrame(np.cumsum(PCA().fit(vector1).explained_variance_ratio_))
pca = PCA(n_components=71)
df1= pd.DataFrame(pca.fit_transform(vector1))

p2=pd.DataFrame(np.cumsum(PCA().fit(vector2).explained_variance_ratio_))
pca = PCA(n_components=138)
df2= pd.DataFrame(pca.fit_transform(vector2))

p3=pd.DataFrame(np.cumsum(PCA().fit(vector3).explained_variance_ratio_))
pca = PCA(n_components=160)
df3= pd.DataFrame(pca.fit_transform(vector3))


In [44]:
'''
# Case1-1 : 데이터를 합쳐서 사용
df=np.concatenate([df1,df2,df3],axis=1)# 71+138+160=369
df=pd.DataFrame(df)
df

'''

,0,1,2,3,4,5,6,7,8,9,...,359,360,361,362,363,364,365,366,367,368
0,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,-0.073251,0.384938,0.210304,-0.585296,1.994303,-0.023081,-0.578174,-0.891166,-0.127750,-0.275945
1,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,1.139345,0.098702,0.488389,0.875926,0.056391,-0.703451,0.593115,1.182725,-0.595985,-0.914268
2,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,-0.143241,-0.295374,0.132397,1.108469,0.201369,0.884499,0.448358,-1.159614,-0.596171,0.906113
3,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,0.719708,-1.923140,-0.161451,0.462175,1.204536,-0.247174,-0.094558,0.472821,-0.379885,-0.144929
4,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,-0.059151,-0.598319,0.447765,-1.891598,0.224196,0.039655,-0.426435,-0.944575,1.151542,-0.591170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,0.002052,0.525898,0.732829,0.235997,0.582471,-0.032896,-0.489716,-0.547669,-0.691931,-0.058891
7609,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,0.420228,-0.663512,1.101274,1.608580,-0.265390,-0.835464,-0.572448,0.493800,-0.447870,0.690452
7610,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,-0.151735,0.609178,1.037368,1.046888,-1.083711,-1.202693,0.880773,-0.505235,0.492676,-1.351583
7611,-4.251546,3.409714,8.77979,-0.228742,0.917307,0.803332,-2.241532,0.144022,2.881155,-2.34727,...,-0.938014,0.727641,-0.336408,-0.364619,-0.265083,0.196129,1.531944,0.234115,-0.569627,-0.389732


In [43]:
'''
DTCmodel=DecisionTreeClassifier(random_state=123).fit(x_train,y_train)
y_pred5=DTCmodel.predict(x_test)

train_score=DTCmodel.score(x_train,y_train)
validation_score=DTCmodel.score(x_val,y_val)

differ=round((train_score-validation_score),3)


if not 0<=differ<=0.05:
    print(differ,'=> 과적합')
else:
    print(differ,'=> 과적합 아님')


print('f1 score =',round(f1_score(y_test,y_pred5),3))
'''

0.293 => 과적합
f1 score = 0.667


In [47]:
#keyword,location 빼고 오로지 text로만 학습
df=pd.DataFrame(df1)

x_train,x_test,y_train,y_test=train_test_split(df,y,train_size=0.7,random_state=123)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, random_state=123)

DTCmodel=DecisionTreeClassifier(random_state=123).fit(x_train,y_train)
y_pred5=DTCmodel.predict(x_test)

train_score=DTCmodel.score(x_train,y_train)
validation_score=DTCmodel.score(x_val,y_val)

differ=round((train_score-validation_score),3)


if not 0<=differ<=0.05:
    print(differ,'=> 과적합')
else:
    print(differ,'=> 과적합 아님')


print('f1 score =',round(f1_score(y_test,y_pred5),3)) 

0.028 => 과적합 아님
f1 score = 0.67


In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score

# 의사결정트리 모델을 위한 하이퍼파라미터 범위 설정
param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [5, 10, 15, 20, None],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10, 20],  # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4, 10],  # 리프 노드에서 최소 샘플 수
    'max_features': [None, 'sqrt', 'log2'],  # 분할 시 고려할 최대 특성 수
}

# RandomizedSearchCV로 대략적인 최적 파라미터를 찾기
g1 = RandomizedSearchCV(DecisionTreeClassifier(random_state=123), param_distributions=param_grid,
                        n_iter=50, cv=5, random_state=42, n_jobs=-1)
g1.fit(x_train, y_train)

# GridSearchCV로 최적의 파라미터를 세밀하게 조정
g2 = GridSearchCV(DecisionTreeClassifier(random_state=123), param_grid=param_grid, cv=5, n_jobs=-1)
g2.fit(x_train, y_train)

# 최적의 파라미터 출력
print("RandomizedSearchCV best params:", g1.best_params_)
print("GridSearchCV best params:", g2.best_params_)

# 최적 파라미터로 모델 학습
best_params_randomized = g1.best_params_
best_params_grid = g2.best_params_

# RandomizedSearchCV 결과로 모델 학습
final_model_randomized = DecisionTreeClassifier(random_state=42, **best_params_randomized)
final_model_randomized.fit(x_train, y_train)

# GridSearchCV 결과로 모델 학습
final_model_grid = DecisionTreeClassifier(random_state=42, **best_params_grid)
final_model_grid.fit(x_train, y_train)

# 예측 수행
y_pred_randomized = final_model_randomized.predict(x_test)
y_pred_grid = final_model_grid.predict(x_test)

# F1-score 계산
f1_randomized = f1_score(y_test, y_pred_randomized, average='weighted')
f1_grid = f1_score(y_test, y_pred_grid, average='weighted')

# 결과 출력
print("RandomizedSearchCV 최적 파라미터로 학습한 모델 F1-score:", round(f1_randomized, 3))
print("GridSearchCV 최적 파라미터로 학습한 모델 F1-score:", round(f1_grid, 3))

# 훈련 데이터와 검증 데이터 점수 차이 계산 (과적합 확인)
train_score_randomized = final_model_randomized.score(x_train, y_train)
validation_score_randomized = final_model_randomized.score(x_val, y_val)

train_score_grid = final_model_grid.score(x_train, y_train)
validation_score_grid = final_model_grid.score(x_val, y_val)

# 점수 차이 계산
differ_randomized = round((train_score_randomized - validation_score_randomized), 3)
differ_grid = round((train_score_grid - validation_score_grid), 3)

# 과적합 여부 확인
if not 0 <= differ_randomized <= 0.05:
    print(f"RandomizedSearchCV 차이 ({differ_randomized}) => 과적합")
else:
    print(f"RandomizedSearchCV 차이 ({differ_randomized}) => 과적합 아님")

if not 0 <= differ_grid <= 0.05:
    print(f"GridSearchCV 차이 ({differ_grid}) => 과적합")
else:
    print(f"GridSearchCV 차이 ({differ_grid}) => 과적합 아님")


RandomizedSearchCV best params: {'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'gini'}
GridSearchCV best params: {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 20}
RandomizedSearchCV 최적 파라미터로 학습한 모델 F1-score: 0.727
GridSearchCV 최적 파라미터로 학습한 모델 F1-score: 0.725
RandomizedSearchCV 차이 (0.027) => 과적합 아님
GridSearchCV 차이 (0.026) => 과적합 아님
